# Imports

In [1]:
from faker import Faker
import pandas as pd

# `OrdinalEncoder`

Comme son nom l'indique, il sert à **encoder** des variables **ordinales**, c'est à dire des variables **catégorielles** pour lesquelles il existe une **hiérarchie parmi les modalités**.

Par exemple, des tailles de vêtements.

## Génération de donnée factice

Nous générons la donnée que nous allons encoder.

In [10]:
fake = Faker()
Faker.seed(69)
num_entries = 10


fake_sizes = [fake.random_element(elements=('XS', 'S', 'M', 'L', 'XL'))
              for _ in range(num_entries)]

sizes = pd.DataFrame({'size': fake_sizes})
sizes

,size
0,XS
1,XS
2,S
3,XS
4,XL
5,M
6,M
7,XL
8,L
9,L


## Implémentation

In [6]:
from sklearn.preprocessing import OrdinalEncoder

In [7]:
# Lister les modalités par ordre croissant
categories = [["XS", "S", "M", "L", "XL"]]  # ATTENTION: liste de listes!

encoder = OrdinalEncoder(categories = categories,
                         # Stratégie en cas de modalité inconnue dans le val ou le test set
                         handle_unknown = "use_encoded_value",
                         # Valeur assignée par défaut à ces modalités inconnues
                         unknown_value = -1)

In [13]:
# Régler l'output du transformer en structures pandas
encoder.set_output(transform="pandas")

sizes_tf = encoder.fit_transform(sizes)

# Check-out de l'encodage
pd.concat([sizes, sizes_tf], axis=1)

,size,size
0,XS,0.0
1,XS,0.0
2,S,1.0
3,XS,0.0
4,XL,4.0
5,M,2.0
6,M,2.0
7,XL,4.0
8,L,3.0
9,L,3.0


In [14]:
# En cas de modalité inconnue dans les nouvelles données
new_sizes = pd.DataFrame({"size": ["M", "S", "XXL", "L"]})
new_sizes_tf = encoder.fit_transform(new_sizes)
pd.concat([new_sizes, new_sizes_tf], axis=1)

,size,size
0,M,2.0
1,S,1.0
2,XXL,-1.0
3,L,3.0
